In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Table of Contents

* [Introduction](#chapter1)
* [Data Preparation](#chapter2)
* [Modeling](#chapter3)
    * [Model 1](#section_3_1)
    * [Model 2](#section_3_2)
    * [Model 3](#section_3_3)
* [Model Evulation](#chapter4)
* [Conclusion](#chapter5)

### Introduction <a class="anchor" id="chapter1"></a>
In this analysis, we would like to predict the price of used cars of 9 brands. We first combined the dataset from 9 brands, factorizing and grouping cetegorical variables and dealing with NA values. 
Then, we used the function called stepAIC which offers stepwise regression for selecting independent variables. We improved the model by log-transformation and removing outliers. Eventually, we reach a final version of the model that yields the R^2 of 0.854.

### Data Preparation <a class="anchor" id="chapter2"></a>

In [ ]:
# library packages
library(dplyr)
library(lattice)
library(ggplot2)
library(GGally)
library(readr)
library(car)
library(MASS)
library(caret) 
# suppressPackageStartupMessages() 

In [ ]:
# importing csv files
audi <- read_csv("../input/used-car-dataset-ford-and-mercedes/audi.csv", show_col_types = FALSE)
bmw <- read_csv("../input/used-car-dataset-ford-and-mercedes/bmw.csv",show_col_types = FALSE)
ford <- read_csv("../input/used-car-dataset-ford-and-mercedes/ford.csv", show_col_types = FALSE)
hyundi <- read_csv("../input/used-car-dataset-ford-and-mercedes/hyundi.csv", show_col_types = FALSE)
merc <- read_csv("../input/used-car-dataset-ford-and-mercedes/merc.csv",show_col_types = FALSE)
skoda <- read_csv("../input/used-car-dataset-ford-and-mercedes/skoda.csv",show_col_types = FALSE)
toyota <- read_csv("../input/used-car-dataset-ford-and-mercedes/toyota.csv",show_col_types = FALSE)
vauxhall <- read_csv("../input/used-car-dataset-ford-and-mercedes/vauxhall.csv",show_col_types = FALSE)
vw <- read_csv("../input/used-car-dataset-ford-and-mercedes/vw.csv",show_col_types = FALSE)

In [ ]:
# combing dataset

df <- bind_rows(list(audi, bmw, ford, hyundi, merc, skoda, toyota, vauxhall, vw), .id="id")
summary(df)

In [ ]:
# data preparing 
# assign the brand name to the combined dataframe

df <- df %>% mutate( brand = case_when(
  id == 1 ~ "audi", 
  id == 2 ~ "bmw", 
  id == 3 ~ "ford",
  id == 4 ~ "hyundi",
  id == 5 ~ "merc",
  id == 6 ~ "skoda",
  id == 7 ~ "toyota",
  id == 8 ~ "vauxhall",
  id == 9 ~ "vw"))

In [ ]:
# regrouping the categorical variables 

df <- df %>% mutate(engineLevel = case_when (
  engineSize < 1 ~ "0",
  engineSize >= 1 & engineSize < 2 ~ "1",
  engineSize >= 2 & engineSize < 3 ~ "2",
  engineSize >= 3 & engineSize < 4 ~ "3",
  engineSize >= 4 & engineSize < 5 ~ "4",
  engineSize >= 5  ~ "5_plus"
))

df <- df %>% mutate(fuelType_re = case_when (
  fuelType == "Electric" | fuelType == "Other" ~ "Elec_other",
  TRUE ~ fuelType
))

In [ ]:
# replacing the missing tax value with mean of tax
df <- df %>% mutate(tax= replace(tax, is.na(tax), mean(tax, na.rm=TRUE)))

In [ ]:
# change the variable to factor 
summary(as.factor(df$fuelType_re))
cols = c("transmission", "fuelType_re","brand", "engineLevel")
df[cols] <- lapply(df[cols], as.factor)
lapply(df[cols], levels)

# droping the unjustifiable data and check the sume of missing data in each column
df <- df[df$transmission != "Other",]
df <- df[df$year <= "2020", ]
colSums(is.na(df))
summary(df)

In [ ]:
# Spliting the dataset into training and testing data
train.size <- 0.8
set.seed(99)
train.index <- sample.int(length(df$price), round(length(df$price)*train.size))
trains <- df[train.index, ]
tests <- df[-train.index, ]
summary(df)

### Modeling <a class="anchor" id="chapter3"></a>

### First Model <a class="anchor" id="section_3_1"></a>

In [ ]:
# We use stepwise selection for varaible selection:
mfull <- lm(price ~ engineSize + year + transmission + brand + mileage + tax + fuelType_re + mpg, data =trains)

# the stepAIC function can successively add and drop predictors to find a model that has lower AIC and higher adjusted R2
step_mfull <- stepAIC(mfull, direction="both")
summary(step_mfull)

We did not drop any variables after running the stepAIC function, but the adjusted r-squared is only 0.7668.

In [ ]:
#Checking regression assumption by diagonstic plots
options(repr.plot.width = 14, repr.plot.height = 14)
par(mfrow=c(2,2))
plot(step_mfull)

**Results of model 1:**

The diagnostic plots show us that some influential points affect our prediction.
Next, we used Cooks' distance will be used for eliminate the outliers. 

The first plot (residuals v.s. fitted values) is a scatterplot between residuals and predicted values. There are some unusual points on the top left corner, and the points are not distributed evenly.

The second plot (normal Q-Q) is a normal probability plot. It will give a straight line if the errors are distributed normally, but the points at the head and tail deviate severely from the straight line. 

The third plot (scale-location) is a scatterplot showing the spread of residuals. Like the first plot, there are some unusual points at the corner, and the points are not distributed evenly.

The fourth plot (residuals vs leverage) help us find the inferential points. Luckily we do not have points lying outside the Cook's distance.

In the following part, we will do the log transformation on price to reduce skewness.

### Second Model <a class="anchor" id="section_3_2"></a>

In [ ]:
# We tried to improve the model by log transformation
# log transformation on price to fit the normality assumption
df$log_price <- log10(df$price)

train.index <- sample.int(length(df$price), round(length(df$price)*train.size))
trains <- df[train.index, ]
tests <- df[-train.index, ]

options(repr.plot.width = 14, repr.plot.height = 7)
par(mfrow=c(1,2))
qqnorm(df$price, main="Normal Q-Q Plot of Price Before Log Transformation");qqline(df$price)
qqnorm(df$log_price, main="Normal Q-Q Plot of Price After Log Transformation");qqline(df$log_price)

In [ ]:
# We use stepwise selection for varaible selection:
mfull2 <- lm(log_price ~ engineSize + year + transmission + brand + mileage + tax + fuelType_re + mpg, data =trains)

# the stepAIC function can successively add and drop predictors to find a model that has lower AIC and higher adjusted R2
step_mfull2 <- stepAIC(mfull2, direction="both")
summary(step_mfull2)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 14)
par(mfrow=c(2,2))
plot(step_mfull2)

**Results of model 2:**

The adjusted r-squared increased from 0.7668 to 0.8754.

According to the Residuals vs Leverage graph, all the diagnostic plots improved, but there are some influential points. 

### Third Model <a class="anchor" id="section_3_3"></a>

In [ ]:
# We tried to improved the model by removing outliers
# We use the general rule of thumb to remove the points that with a Cook's D of more than 3 times the mean,
cooksD1 <- cooks.distance(step_mfull2)
inf_1 <- cooksD1[(cooksD1 > (3 * mean(cooksD1, na.rm = TRUE)))] 

inf_name1 <- names(inf_1)
outliers1 <- trains[inf_name1,]
trains_after <- trains %>% anti_join(outliers1)

new_mfull <- lm(log_price ~ engineSize + year + transmission + brand + mileage + tax + fuelType_re + mpg, data = trains_after)

step_mfull3 <- stepAIC(new_mfull, direction="both")
summary(step_mfull3)

options(repr.plot.width = 14, repr.plot.height = 14)
par(mfrow=c(2,2))
plot(step_mfull3)

**Results of model 3:**

After removing outliers, the adjusted r-squared increased a little from 0.8754 to 0.8957.

We can see that the points are spread randomly on the graphs from the Residual vs Fitted plot and Scale-Location plot.
The qq plot shows that most residuals follow the straight line, and the deviated points are gone.
Also, for the Residuals-Leverage plot, no more points are lying outside the dashed line. 

### Model Evaluation <a class="anchor" id="chapter4"></a>

In [ ]:
# evaluating the model by the test data
tests$pred_price1  <- predict(step_mfull, newdata=tests, select=c(price, engineSize, year, 
                        transmission, brand, mileage, tax, fuelType_re, mpg))

tests$pred_price_log2  <- predict(step_mfull2, newdata=tests, select=c(log_price, engineSize, year, 
                        transmission, brand, mileage, tax, fuelType_re, mpg))

tests$pred_price_log3 <- predict(step_mfull3, newdata=tests, select=c(log_price, engineSize, year, 
                        transmission, brand, mileage, tax, fuelType_re, mpg))


tests$pred_price2 <- 10^(tests$pred_price_log2)
tests$pred_price3 <- 10^(tests$pred_price_log3)

data.frame(R2_m1 = R2(tests$pred_price1, tests$price),
           RMSE_m1 = RMSE(tests$pred_price1, tests$price),
           MAE_m1 = MAE(tests$pred_price1, tests$price),
           R2_m2 = R2(tests$pred_price2, tests$price),
           RMSE_m2 = RMSE(tests$pred_price2, tests$price),
           MAE_m2 = MAE(tests$pred_price2, tests$price),
           R2_m3 = R2(tests$pred_price3, tests$price),
           RMSE_m3 = RMSE(tests$pred_price3, tests$price),
           MAE_m3 = MAE(tests$pred_price3, tests$price)
          )

In [ ]:
# we check the collinearity by vif function in car package:
vif(step_mfull3)

Since all the GVIF scores are lower than 5, we don't have a collinear problem in this model.

### Conclusion <a class="anchor" id="chapter5"></a>

The third model has a R^2 score of 85.4%, and the RMSE and MAE are lower than others. Thus we will use the third model for price prediction.  

In [ ]:
# predicting car price
pred_try1 <- data.frame(year=2019, transmission = "Automatic", mileage = 20000, mileage=1000,
                        fuelType_re = "Hybrid", tax=100, mpg = 100, engineLevel = "2", engineSize=4, brand="bmw")

10^(predict(step_mfull3, newdata=pred_try1))